<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    words = words.upper()

    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    #チルダ統一1
    ser_word = ["〜","～", "-","－","から","MAX"]
    for word in ser_word:
      words = words.replace(word,"~")

    #稼働時間削除
    if len(re.findall('[0-9]{3}H~[0-9]{3}H',words)) > 0:
      kado = re.findall('[0-9]{3}H~[0-9]{3}H',words)[0]
      words = words.replace(kado,"")
    if len(re.findall('[0-9]{3}~[0-9]{3}H',words)) > 0:
      kado = re.findall('[0-9]{3}~[0-9]{3}H',words)[0]
      words = words.replace(kado,"")
    if len(re.findall('[0-9]{3}~[0-9]{3}',words)) > 0:
      kado = re.findall('[0-9]{3}~[0-9]{3}',words)[0]
      words = words.replace(kado,"")
      
    #スキル見合い
    ser_word = ["スキル見合い","スキル見合","ご相談","相談","ご提示下さい","ご提示ください","スキルにより相談"]
    for word in ser_word:
      words = words.replace(word,"S00")
      
    #曖昧単価加工
    if len(re.findall('[0-9]{1}・',words)) > 0:
      aimai = re.findall('[0-9]{1}・',words)[0]
      words = words.replace(aimai,"")
    if len(re.findall('[0-9]{1}･',words)) > 0:
      aimai = re.findall('[0-9]{1}･',words)[0]
      words = words.replace(aimai,"")
    if len(re.findall('[0-9]{1}、',words)) > 0:
      aimai = re.findall('[0-9]{1}、',words)[0]
      words = words.replace(aimai,"")
      
    #改行コード変換
    words = words.replace("\r\n","")

    #小数点無効
    words = words.replace(".0","")

    #万円単位変換
    words = words.replace("0000","")
      
    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】","(",")","（","）"," ","　",":","：","完全","位","くらい","ぐらい","およそ","大体","だいたい","約","前後","円","万","下さい","ください"]
  
  base_word = '単価'
  tanka_yuragi = ["予算単価","単金","金額","価格","月額","予算"]

  #◆単価が含まれるであろう箇所だけ渡す
  #正規化
  str_wk = serialize(str_wk)

  #削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')

  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)

  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「単価」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""

  #上限範囲指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+1] == "~":
      tanka_str = "00~" + list_output[list_output.index(base_word)+2]

  #上限下限指定
  #if list_output.count(base_word) == 1:
  #  if list_output[list_output.index(base_word)+2] == "~":
  #    tanka_str = tanka_str + "~" + list_output[list_output.index(base_word)+3]

  #上限のみ指定
  #★エラー有り
  #if list_output[list_output.index(base_word)+1] == "MAX":
  #  tanka_str ="00~" + list_output[list_output.index(base_word)+2]
    
  if list_output.count(base_word) > 0:
    if list_output[list_output.index(base_word)+1].isdecimal() is True or list_output[list_output.index(base_word)+2].isdecimal() is True or list_output[list_output.index(base_word)+3].isdecimal() is True:
      tanka_str = list_output[list_output.index(base_word)+1] + "/" + list_output[list_output.index(base_word)+2] + "/" + list_output[list_output.index(base_word)+3] + "/" +list_output[list_output.index(base_word)+4]
    else:
      tanka_str = "単価情報あり"
  else:
    tanka_str = "単価完全なし"
    
  return tanka_str

In [125]:
import pandas as pd
import codecs

#utf-8で作られていないファイルの場合のCSV読み込み方法
with codecs.open("python.CSV", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

df2 = df[["本文"]]

#行単位の繰り返し
for index,item in df2.iterrows():
  str_wk = item['本文']
  pickup_str = tanka_pickup(str_wk)
  print(str(index) + ":" + pickup_str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


0:~/55///月
1:単価完全なし
2:単価完全なし
3:単価完全なし
4:単価情報あり
5:65/◆/枠/数
6:~/85/◆/面談
7:80/まで/S/00
8:80///月/、
9:~/90/◆/面談
10:60/~/80/精算
11:単価完全なし
12:~/75/◆/支払
13:85/◆/面談/2
14:~/95/◆/面談
15:~/95/◆/面談
16:S/00/できる/方
17:~/70/◆/支払
18:80///月/、
19:60/~/80/精算
20:S/00/できる/方
21:~/85/◆/面談
22:65/◆/枠/数
23:~/90/◆/面談
24:~/90/◆/面談
25:80/~/90/程度
26:80///月/、
27:S/00/※/PYTHON
28:85/◆/面談/2
29:~/95/◆/面談
30:~/95/◆/面談
31:~/75/◆/支払
32:~/70/◆/支払
33:80/~/90/程度
34:85/◆/面談/2
35:S/00/※/PYTHON
36:~/95/◆/面談
37:~/95/◆/面談
38:S/00/応/S
39:~/75/◆/支払
40:90/まで/※/S
41:~/70/◆/支払
42:85/まで/※/S
43:54/業種/職務/システムエンジニア
44:単価完全なし
45:S/00/※/PYTHON
46:~/85/◆/面談
47:65/◆/枠/数
48:~/75/S/00
49:S/00/■/精算
50:80/まで/S/00
51:~/90/◆/面談
52:S/00/応/S
53:60/~/80/精算
54:85/◆/面談/2
55:~/95/◆/面談
56:~/95/◆/面談
57:65/~/75/精算
58:~/75/◆/支払
59:~/70/◆/支払
60:~~/150/※/S
61:65/◆/枠/数
62:60/~/80/※
63:60/~/80/精算
64:70/~※/S/00
65:S/00/※/ご
66:85/◆/面談/2
67:60/~/80/※
68:S/00/できる/方
69:~/75/◆/支払
70:~/70/◆/支払
71:90/まで/※/PYTHON
72:~/95/◆/面談
73:~/95/◆/面談
74:65/~※/S/00
75:80/~/90/程度
76:~~/150

In [0]:
print(df2.iloc[166])

本文    \r\n\r\n-----Original Message-----\r\nFrom: 【ス...
Name: 166, dtype: object


In [0]:
#単価変数初期化
min_tanka = 0
max_tanka = 0
med_tanka = 0

#======================================================
#範囲指定単価(exp:A～B) 編集
#======================================================
tanka_range = re.findall('[0-9]{2}~[0-9]{2}',pickup_str)

#中間値算出
#if len(tanka_range) == 1:
#  tanka1 = int(tanka_range[0].split('~')[0])
#  tanka2 = int(tanka_range[0].split('~')[1])
#  med_tanka = round((tanka1+tanka2)/2)
#else:
#  med_tanka = "0"

#下限算出
if len(tanka_range) == 1:
  min_tanka = int(tanka_range[0].split('~')[0])
else:
  min_tanka = 0

#上限算出
if len(tanka_range) == 1:
  max_tanka = int(tanka_range[0].split('~')[1])
else:
  max_tanka = 0

print("MIN:" + str(min_tanka))
print("MAX:" + str(max_tanka))


00~55
MIN:0
MAX:55
